# Easy Agent Tutorial
This notebook file provide three examples of using LLM based agents with different tool sets.

Prerequisites:
- Python 3.10+
- Install required packages:
  ```bash
  pip install requests python-dotenv langchain-community langchain-openai langchain langgraph "smolagents[mcp, gradio]" faiss-cpu tenacity pyyaml
  ```

## Task Description

如README所述，该项目应用三种方案，从不同的角度实现了agentic RAG的功能。为了演示，这一次我们将会构建一个信安四大会的查询，来进行感兴趣论文的搜索以及基于题目选择合适的会议进行投稿。

## Data Preparation

In [ ]:
# 我们将使用dblp数据集来进行演示。首先下载信安四大会2025年的会议论文数据：
import requests
import json
import os

if not os.path.exists("dblp_sec_papers.json"):
    with requests.Session() as sess:
        url = "https://dblp.org/search/publ/api"
        params = {"q": "security", "format": "json"}
        tocs = {
            "ndss": "toc:db/conf/ndss/ndss2025.bht:",
            "sp": "toc:db/conf/sp/sp2025.bht:",
            "ccs": "toc:db/conf/ccs/ccs2025.bht:",
            "usenix": "toc:db/conf/uss/uss2025.bht:",
        }
        papers = []
        for k, v in tocs.items():
            response = sess.get(url, params={"q": v, "h": 1000, "format": "json"})
            data = response.json()
            data = data["result"]["hits"]["hit"]
            papers.extend(data)
        with open(f"dblp_sec_papers.json", "w", encoding="utf-8") as f:
            json.dump(papers, f, ensure_ascii=False, indent=4)
else:
    with open(f"dblp_sec_papers.json", "r", encoding="utf-8") as f:
        papers = json.load(f)
papers = [x["info"] for x in papers]
titles = [f"[{x['venue']} {x['year']}] {x['title']}" for x in papers]

在进行下一步之前，需要对环境进行一些配置，
创建一个`.env`文件，并添加以下内容：

-   示例 1：OpenAI 官方服务

```
OPENAI_API_KEY=your_openai_api_key
HTTPS_PROXY=your_proxy
MODEL_NAME=gpt-5.1
EMB_MODEL_NAME=text-embedding-3
```

In [5]:
# 创建并保存向量数据库
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
if "db" not in globals():
    db = FAISS.from_texts(titles, OpenAIEmbeddings())
db.save_local("faiss_db")

## 方案0： 一切奇迹的始发点——传统工具调用

在开始之前，先看一下传统的工具调用大概长啥样，有怎样的优缺点

In [ ]:
import json
import logging
import re

import dotenv
import tenacity
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from openai.types import *
from openai.types.chat import *

load_dotenv()


def raw_toolcall():
    client = OpenAI()
    db = FAISS.load_local(
        "faiss_db",
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True,
    )
    tools_def = [
        {
            "type": "function",
            "name": "query_paperdb",
            "description": "accept keyword and return related information",
            "parameters": {
                "type": "object",
                "properties": {
                    "kw": {
                        "type": "string",
                        "description": "The keyword to query the database",
                    }
                },
                "required": ["kw"],
            },
        }
    ]

    def real_ask(question: str):
        from openai.types.responses.response_input_param import Message

        input_msgs: list[Message] = [
            {
                "type": "message",
                "role": "system",
                "content": f"You are a helpful research assistant. When given a question, you must first decide if you need to query the database to get relevant information. If so, use the tool 'query_paperdb' with appropriate keywords extracted from the question. After getting the information, provide a comprehensive answer based on both the retrieved information and your own knowledge.",
            },
            {
                "type": "message",
                "role": "user",
                "content": question,
            },
        ]
        resp = client.responses.create(
            model="gpt-5.1",
            tools=tools_def,
            input=input_msgs,
            tool_choice="required",
        )
        for toolcall in resp.output:
            if toolcall.type != "function_call":
                continue
            if toolcall.name == "query_paperdb":
                kw = json.loads(toolcall.arguments)["kw"]
                print(f"Keyword: {kw}")
                docs = []
                for skw in kw.split():
                    if not (skw := skw.strip()):
                        continue
                    print(f"Searching for keyword: {skw}")
                    docs.extend(db.similarity_search(skw, k=30))
                rag_result = "\n".join([doc.page_content for doc in docs])
                input_msgs.append(toolcall)
                input_msgs.append(
                    {
                        "type": "function_call_output",
                        "call_id": toolcall.call_id,
                        "output": str(rag_result),
                    }
                )
                print(f"{kw=} appended.")
        if input_msgs[-1]["type"] == "function_call_output":
            resp = client.responses.create(
                model="gpt-5.1",
                input=input_msgs,
                # tools=tools_def,
                stream=True,
            )
            for chunk in resp:
                if chunk.type == "response.output_text.delta":
                    print(chunk.delta, end="", flush=True)
            print()
        else:
            print(resp.output_text)

    while True:
        inp = input("=> ")
        if not inp.strip():
            break
        real_ask(inp)


raw_toolcall()

Keyword: llm安全 论文
Searching for keyword: llm安全
Searching for keyword: 论文
kw='llm安全 论文' appended.
下面给你整理一批近两年「LLM 安全」相关的代表性论文，主要来自 USENIX Security / IEEE S&P (Oakland) / CCS / NDSS 2025，方便你查阅和做文献综述。我先按主题简单分一下类，再列核心论文（中文说明 + 原题方便你搜索）。

如果你说明更具体的方向（比如：越狱攻击、RAG 安全、隐私泄露、模型水印、医疗场景等），我可以帮你做更细的分类和阅读笔记。

---

## 一、LLM 越狱与对抗攻击 / 防御

**攻击与基准**

1. **Exploiting Task-Level Vulnerabilities: An Automatic Jailbreak Attack and Defense Benchmarking for LLMs**  
   - 会议：USENIX Security 2025  
   - 关键词：自动化越狱、任务级脆弱性、攻击与防御基准

2. **PAPILLON: Efficient and Stealthy Fuzz Testing-Powered Jailbreaks for LLMs**  
   - 会议：USENIX Security 2025  
   - 用模糊测试（fuzzing）自动探索提示空间，实现高效隐蔽的越狱。

3. **Mind the Inconspicuous: Revealing the Hidden Weakness in Aligned LLMs' Refusal Boundaries**  
   - 会议：USENIX Security 2025  
   - 分析对齐后模型拒答边界中“不起眼”的薄弱点，属于越狱与安全对齐交叉。

4. **Fuzz-Testing Meets LLM-Based Agents: An Automated and Efficient Framework for Jailbreaking Text-to-Image Generation Models**  
   - 会议：IEEE S&P (SP) 2025  
   -

### 传统工具调用的优缺点

- 优点：直接用模型原生的 function/tool 调用协议，链路短、开销低，JSON Schema 参数校验清晰。
- 优点：可以精确控制何时调用工具、使用 `tool_choice` 等参数强制执行，消息格式透明、便于调试和流式输出。
- 优点：依赖少，不绑框架，易于插入到现有服务或与其他编排层组合。
- 缺点：需要手写对话状态管理、工具输入输出拼接，容易出错且样板代码多。
- 缺点：缺少自动规划/多步推理、重试、fallback 等封装能力，复杂流程要自行实现。
- 缺点：与特定模型/协议耦合，换提供方或多模型时需适配；安全性与数据清洗（如反序列化、去重）也要自管。

## 方案1： SmolAgent::CodeAgent

SmolAgent::CodeAgent 拥有两种提示模板，使用`use_structured_outputs_internally`参数进行控制。
- 核心循环：两条线路都遵循多步推理，但输出格式不同。
  - 标准版（`code_agent.yaml`，`use_structured_outputs_internally=False`）：每步输出 Thought→Code→Observation，代码必须包裹在自定义 `{{code_block_opening_tag}}`/`{{code_block_closing_tag}}` 中；中间结果用 `print` 进入 Observation，最终用 `final_answer` 收束。
  - 结构化版（`structured_code_agent.yaml`，`use_structured_outputs_internally=True`）：每步输出固定 JSON `{ "thought": "...", "code": "..." }`，默认不需要自定义 code_block 标签；解析稳定、便于日志/回放。
- 规划 scaffold：两者都要求先做 facts survey + high-level plan（initial_plan / update_plan），但标准版在文本里更强调「分步打印、避免链式依赖」。
- 工具调用与链式策略：
  - 标准版明确区分“有 JSON schema 的工具可以链式调用，非结构化工具应先 `print` 再下一步用”，并警示不要重做相同参数的调用、不要用工具名做变量名。
  - 结构化版主要提供最小约束（定义变量再用、参数名显式），输出以 JSON 承载 Thought 和 Code，便于上层程序直接消费。
- 执行与可读性：
  - 标准版输出包含 Thought/Code/Observation 叙事，适合人工旁观调试、流式展示和教学场景。
  - 结构化版输出紧凑、机器友好，适合日志解析、监控、对接编排器或二次路由。
- 选择建议：
  - 需要确定性结构、便于程序解析/回放/审计时，用 `use_structured_outputs_internally=True`（结构化版）。
  - 需要人类可读的逐步对话、希望看到 code_block 包裹和 Observation 明细，或想依赖模板中的链式提示时，用 `use_structured_outputs_internally=False`（标准版）。
- 常见踩坑提示：
  - 标准版务必保持自定义 code_block 标签，否则解析失败；非结构化工具调用不要在同一块紧接依赖下一工具输出。
  - 结构化版的 JSON 输出中请将可执行代码写在 `code` 字段，仍需显式 `final_answer(...)` 收口；确保不要把工具名当变量名。
- 性能与上下文：标准版提示体积更大，可能略增 token 开销；结构化版更紧凑，节省上下文。

In [ ]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from smolagents import CodeAgent, OpenAIServerModel, tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)

sys_prompt = """
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
Output the final answer in markdown wrapped in final_answer().
""".strip()


@tool
def query_paperdb(kw: str) -> str:
    """
    Accept keywords and return related paper titles, multiple keywords should be separated by '|'.
    This tool should be called before output any realworld-related information.

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split("|"):
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=10))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


agent = CodeAgent(
    model=OpenAIServerModel("gpt-5.1"),
    tools=[query_paperdb],
    stream_outputs=True,
    # use_structured_outputs_internally=True, # True for structured_code_agent.yaml, False for code_agent.yaml
)

# from Gradio_UI import GradioUI

# GradioUI(agent).launch()

_ = agent.run(f"{sys_prompt}\n\n{input("=> ")}", return_full_result=True)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a helpful research assistant.                                                                           │
│ When given a question, you must query the database to get relevant information.                                 │
│ Use the tools with appropriate arguments derived from the question.                                             │
│ After getting the information, provide a comprehensive answer based on both the retrieved information.          │
│ Output the final answer in markdown wrapped in final_answer().                                                  │
│                                                                                                                 │
│ 查询llm安全相关的论文                                                                                           │
│                                                                                                                 │
╰─ OpenAIModel - gpt-5.1 ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from typing import List                                                                                          
                                                                                                                   
  # Step 1: Query the paper database for LLM safety related keywords (in English and Chinese)                      
  kw = "LLM safety|large language model safety|AI alignment|红队 攻击 LLM|大模型 安全 防护|jailbreak defense"      
  papers_info = query_paperdb(kw=kw)                                                                               
                                                                                                                   
  print(papers_info)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'from typing import List' due to: InterpreterError: Import from typing is not 
allowed. Authorized imports are: ['queue', 'collections', 'unicodedata', 'datetime', 'stat', 'random', 're', 
'math', 'statistics', 'time', 'itertools'\]

...

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # 这里我们已经拿到了大量与“大语言模型（LLM）安全”高度相关的论文题目                                              
  # 接下来不再调用工具，而是基于这些结果，按主题给你做一个中文综述与阅读建议。                                     
                                                                                                                   
  summary = r"""                                                                                                   
  下面根据你查询到的 llm 安全相关论文列表，按研究方向给出一个结构化综述和阅读建议。                                
  （会议缩写：SP=IEEE S&P，CCS=ACM CCS，NDSS=NDSS，USENIX=USENIX Security）                                        
                                                                                                                   
  ---                                                                                                              
                                                                                                                   
  ## 一、总体图景：LLM 安全/对齐的主要研究方向                                                                     
                                                                                                                   
  从检索结果看，LLM 安全论文大致可以分为几大类：                                                                   
                                                                                                                   
  1. **越狱（Jailbreak）与红队攻击**                                                                               
     - 设计更强的越狱攻击方法，系统性评估主流模型的防御能力。                                                      
     - 提出自动化测试/模糊测试（fuzzing）等技术来发现安全策略的弱点。                                              
     - 同时研究防御策略，例如对齐增强、概念级控制等。                                                              
                                                                                                                   
  2. **提示注入（Prompt Injection）、Prompt 窃取和推断**                                                           
     - 针对多轮对话、RAG、代理（agent）、提示服务等场景下的注入攻击。                                              
     - 提出结构化查询、对齐优化等防御机制。                                                                        
     - 研究 Prompt Stealing / Prompt Inference 等推断攻击。                                                        
                                                                                                                   
  3. **对齐与“安全失配”（Safety Misalignment）**                                                                   
     - 模型尽管被对齐，但在复杂场景中仍会出现不安全输出。                                                          
     - 分析对齐在微调、工具调用、RAG 等场景中的退化问题。                                                          
     - 探讨如何在保持能力的同时，减少安全对齐的“遗失”。                                                            
                                                                                                                   
  4. **隐私与数据泄露**                                                                                            
     - 从模型中提取个人敏感信息（PII）。                                                                           
     - 成员推断（membership inference）与训练数据保护。                                                            
     - 在医疗等敏感领域评估 LLM 的隐私风险与防御。                                                                 
                                                                                                                   
  5. **后门、模型合并与生成数据风险**                                                                              
     - 大模型后门攻击与检测方法。                                                                                  
     - 模型合并（model merging）中的隐含安全风险。                                                                 
     - 使用生成数据微调可能引入伪隐私、偏见和安全问题。                                                            
                                                                                                   

Final answer: 
下面根据你查询到的 llm 安全相关论文列表，按研究方向给出一个结构化综述和阅读建议。  
（会议缩写：SP=IEEE S&P，CCS=ACM CCS，NDSS=NDSS，USENIX=USENIX Security）

---

## 一、总体图景：LLM 安全/对齐的主要研究方向

从检索结果看，LLM 安全论文大致可以分为几大类：

1. **越狱（Jailbreak）与红队攻击**
   - 设计更强的越狱攻击方法，系统性评估主流模型的防御能力。
   - 提出自动化测试/模糊测试（fuzzing）等技术来发现安全策略的弱点。
   - 同时研究防御策略，例如对齐增强、概念级控制等。

2. **提示注入（Prompt Injection）、Prompt 窃取和推断**
   - 针对多轮对话、RAG、代理（agent）、提示服务等场景下的注入攻击。
   - 提出结构化查询、对齐优化等防御机制。
   - 研究 Prompt Stealing / Prompt Inference 等推断攻击。

3. **对齐与“安全失配”（Safety Misalignment）**
   - 模型尽管被对齐，但在复杂场景中仍会出现不安全输出。
   - 分析对齐在微调、工具调用、RAG 等场景中的退化问题。
   - 探讨如何在保持能力的同时，减少安全对齐的“遗失”。

4. **隐私与数据泄露**
   - 从模型中提取个人敏感信息（PII）。
   - 成员推断（membership inference）与训练数据保护。
   - 在医疗等敏感领域评估 LLM 的隐私风险与防御。

5. **后门、模型合并与生成数据风险**
   - 大模型后门攻击与检测方法。
   - 模型合并（model merging）中的隐含安全风险。
   - 使用生成数据微调可能引入伪隐私、偏见和安全问题。

6. **面向特定群体/应用的安全**
   - 面向青少年/未成年人内容安全。
   - 面向代码生成、漏洞检测等安全相关任务的 LLM 应用。
   - 面向搜索、Web 访问、应用商店等“LLM 上线”场景的新风险。

7. **多模态安全（Vision-Language / Text-to-Image）**
   - 图文大模型识别不安全内容的能力评估与改进。
   - 文本到图像模型的越狱与防御。

下面以具体论文为例，给你一个“按方向+代表论文”的阅读清单和简要要点，方便你搭建知识体系。

---

## 二、越狱攻击与防御（Jailbreak / Red-teaming）

**攻击方法类：**

- **[USENIX 2025] TwinBreak: Jailbreaking LLM Security Alignments based on Twin Prompts**  
  - 思路：利用“成对的双胞胎提示（Twin Prompts）”构造更隐蔽、稳定的越狱攻击。
  - 贡献：系统展现现有对齐策略如何在 Twin Prompts 下被绕过，说明安全策略对某些结构化提示不够鲁棒。
  - 你可以关注：提示模式（pattern），对齐策略在结构对称下的失效机理。

- **[USENIX 2025] PAPILLON: Efficient and Stealthy Fuzz Testing-Powered Jailbreaks for LLMs**  
  - 思路：将“模糊测试（fuzzing）”思想引入越狱提示搜索，自动生成多样化攻击提示。
  - 特点：高效率、可自动运行，适合作为安全评估工具链的一部分。

- **[USENIX 2025] Great, Now Write an Article About That: The Crescendo Multi-Turn LLM Jailbreak Attack**  
  - 思路：多轮对话式越狱，先从温和问题逐渐诱导模型突破安全边界。
  - 价值：提示多轮交互引入的新攻击面，单轮审查防御不足。

- **[SP 2025] Fuzz-Testing Meets LLM-Based Agents: An Automated and Efficient Framework for Jailbreaking 
Text-to-Image Generation Models**  
  - 虽是图像生成模型，但方法论（代理+fuzz）可迁移到 LLM 红队自动化。

**防御方法类：**

- **[USENIX 2025] SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical Manner**  
  - 核心：利用 LLM 自身能力构造自我防御管线（如自审查、自修正）。
  - 实践性：强调可部署性，对工程系统启发大。

- **[USENIX 2025] JBShield: Defending Large Language Models from Jailbreak Attacks through Activated Concept 
Analysis and Manipulation**  
  - 关键点：通过“激活概念（activated concepts）”视角理解和控制模型内部对有害内容的表征。
  - 特点：更偏向可解释+内部干预，而非仅靠外部规则过滤。

- **[USENIX 2025] Exploiting Task-Level Vulnerabilities: An Automatic Jailbreak Attack and Defense Benchmarking for
LLMs**  
  - 贡献：不只给攻击，还给出一套评价不同防御配置的基准框架，适合做横向对比实验。

---

## 三、提示注入、提示窃取与结构化防御

**攻击与测量：**

- **[SP 2025] On the Effectiveness of Prompt Stealing Attacks on In-the-Wild Prompts**  
- **[USENIX 2025] PRSA: Prompt Stealing Attacks against Real-World Prompt Services**  
  - 聚焦：通过反向推理、黑盒查询等方式从在线服务中窃取高价值系统提示（system prompt / jailbreak prompt）。
  - 有助于理解 Prompt 作为“知识资产”的威胁模型。

- **[SP 2025] Prompt Inversion Attack Against Collaborative Inference of Large Language Models**  
- **[CCS 2025] Prompt Inference Attack on Distributed Large Language Model Inference Frameworks**  
  - 场景：多方推理、分布式推理中的 prompt 泄露风险。

- **[SP 2025] Fun-tuning: Characterizing the Vulnerability of Proprietary LLMs to Optimization-Based Prompt 
Injection Attacks via the Fine-Tuning Interface**  
  - 看点：利用“微调接口”实现类似 prompt injection 的效果，说明开放微调 API 本身就是一个新的攻击面。

**防御与对齐优化：**

- **[USENIX 2025] StruQ: Defending Against Prompt Injection with Structured Queries**  
  - 思路：将自然语言指令映射为结构化查询/调用，减少模型对“任意文本”的被动执行，限制攻击面。

- **[CCS 2025] SecAlign: Defending Against Prompt Injection with Preference Optimization**  
  - 方法：通过偏好优化（preference optimization）训练模型拒绝 prompt injection 类的恶意模式。

- **[SP 2025] DataSentinel: A Game-Theoretic Detection of Prompt Injection Attacks**  
  - 视角：博弈论建模攻击者/防御者，从策略优化角度设计检测机制。

---

## 四、安全对齐与“(In)Security / Misalignment”问题

- **[NDSS 2025] Safety Misalignment Against Large Language Models**  
  - 重点：系统性分析模型在不同任务/场景下的安全表现与对齐目标的偏差。
  - 有利于理解“为什么看起来合规的模型在某些边缘情况仍危险”。

- **[SP 2025] Alleviating the Fear of Losing Alignment in LLM Fine-tuning**  
  - 关注：下游微调是否、以及如何破

[Step 3: Duration 53.20 seconds| Input tokens: 10,080 | Output tokens: 4,046]

res.token_usage=TokenUsage(input_tokens=10080, output_tokens=4046, total_tokens=14126) res.timing=Timing(start_time=1765350604.0020978, end_time=1765350669.0835783, duration=65.08148050308228)


## 方案2： LangGraph

以下代码块用 LangGraph 重新实现「检索→工具调用→回答」的代理流程，强调显式状态机与可视化路由的优势：



- 基础资源：启动时加载 `.env`，用 `FAISS.load_local` + `OpenAIEmbeddings` 复用前面构建的向量库，确保跨会话复现检索结果。

- 系统提示：`sys_prompt` 要求回答前必须调用检索工具，并用 `final_answer(...)` 收口；保持与前两种方案一致，便于对比 token 与行为。

- 工具实现：`@tool` 装饰的 `query_paperdb` 按 `|` 分割多关键词，每个关键词检索 top-10，stdout 打印检索日志，返回拼接文本作为 RAG 证据。

- 模型与绑定：`ChatOpenAI(model="gpt-5.1", verbose=True)` 绑定工具得到 `chat_with_tools`，LangGraph 自动根据模型给出的 `tool_calls` 决定是否进入工具节点。

- 状态与路由：`AgentState` 仅维护 `messages`（用 `add_messages` 合并），`StateGraph` 定义两个节点：
  - `assistant`：调用大模型，可能产出工具调用或直接回答。
  - `tools`：`ToolNode` 执行 `query_paperdb` 并把结果写回消息。
 通过 `tools_condition` 条件边在两节点间循环，直到模型不再请求工具。

- 编译与运行：`builder.compile()` 得到 `agent`，随后用 `SystemMessage+HumanMessage` 作为初始消息调用 `agent.invoke`。执行结束后按顺序打印每段回复，便于观察模型与工具交替的对话片段。

- 适用场景：当需要显式控制多轮工具路由、插入额外节点（如过滤、重排序、裁剪上下文）或做流程可视化/监控时，LangGraph 方案更易扩展；相较裸工具调用（方案0）和 SmolAgent（方案1），它在可编排性与可观测性上更强。

In [ ]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools import tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)

sys_prompt = """
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
Output the final answer in markdown wrapped in final_answer().
""".strip()


@tool
def query_paperdb(kw: str) -> str:
    """
    Accept keywords and return related paper titles, multiple keywords should be separated by '|'.
    This tool should be called before output any realworld-related information.

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split("|"):
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=10))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


from typing import Annotated, TypedDict

from langchain_core.messages import AIMessage, AnyMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

chat = ChatOpenAI(model="gpt-5.1", verbose=True)
chat_with_tools = chat.bind_tools([query_paperdb])


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }


builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode([query_paperdb]))
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

agent = builder.compile()

messages = [
    SystemMessage(content=sys_prompt),
    HumanMessage(content=input("=> ")),
]

response = agent.invoke({"messages": messages})

for idx, part in enumerate(response["messages"]):
    print(f"Part {idx}: \n{part.content}\n\n")

Searching for keyword: LLM安全
Searching for keyword: large language model security
Searching for keyword: prompt injection
Searching for keyword: jailbreak defense
Searching for keyword: model alignment
Searching for keyword: AI安全
Part 0: 
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
Output the final answer in markdown wrapped in final_answer().


Part 1: 
查询llm安全相关的论文


Part 2: 



Part 3: 
[SP 2025] On the (In)Security of LLM App Stores.
[USENIX Security Symposium 2025] When LLMs Go Online: The Emerging Threat of Web-Enabled LLMs.
[CCS 2025] Can Personal Health Information Be Secured in LLM? Privacy Attack and Defense in the Medical Domain.
[USENIX Security Symposium 2025] SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practica

## 方案3： CodeAgent + MCP (Model Context Protocol)

这一节展示如何通过 MCP 将外部工具集合挂载给 SmolAgent 的 CodeAgent，并保持与前面方案一致的“先检索后回答”规范。使用前请先在本目录启动 `mcp-1.py` 以提供本地 MCP 服务器。



- 启动服务：在终端运行 `python mcp-1.py`，保持进程存活，默认监听 `http://127.0.0.1:8000/mcp`。

- 工具获取：`ToolCollection.from_mcp` 以 streamable-http 方式连接服务器，`trust_remote_code=True` 允许加载远端工具实现，`structured_output=False` 让工具返回原始文本。

- 代理配置：创建 `CodeAgent(model=OpenAIServerModel("gpt-5.1"), tools=[*tool_collection.tools], stream_outputs=True, use_structured_outputs_internally=True)`，沿用 `sys_prompt` 强制先调用工具，再用 `final_answer(...)` 收口。

- 运行流程：读取用户输入后直接 `agent.run`，中间的工具调用与结果交由 MCP 服务提供，实现与本地代码解耦、便于统一暴露多工具。

- 适用场景：需要集中托管工具、复用跨项目/多语言工具链，或将工具部署为独立服务时，MCP 能作为标准化桥梁，保持模型侧最小改动。

In [1]:
from smolagents import CodeAgent, OpenAIServerModel, ToolCollection

from dotenv import load_dotenv

load_dotenv()

sys_prompt = """
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
* Output the final answer in markdown wrapped in final_answer().
""".strip()

with ToolCollection.from_mcp(
    {"url": "http://127.0.0.1:8000/mcp", "transport": "streamable-http"},
    trust_remote_code=True,
    structured_output=False,
) as tool_collection:
    agent = CodeAgent(
        model=OpenAIServerModel("gpt-5.1"),
        tools=[*tool_collection.tools],
        stream_outputs=True,
        use_structured_outputs_internally=True,
    )

    # from Gradio_UI import GradioUI

    # GradioUI(agent).launch()

    agent.run(f"{sys_prompt}\n\n{input('=> ')}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a helpful research assistant.                                                                           │
│ When given a question, you must query the database to get relevant information.                                 │
│ Use the tools with appropriate arguments derived from the question.                                             │
│ After getting the information, provide a comprehensive answer based on both the retrieved information.          │
│ * Output the final answer in markdown wrapped in final_answer().                                                │
│                                                                                                                 │
│ 查询llm安全相关的论文                                                                                           │
│                                                                                                                 │
╰─ OpenAIModel - gpt-5.1 ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  papers = query_paperdb("LLM security|large language model security|prompt injection|model stealing|data          
  poisoning|jailbreak attacks")                                                                                    
  print(papers)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'papers = query_paperdb("LLM security|large language model security|prompt 
injection|model stealing|data poisoning|jailbreak attacks")' due to: ValueError: tool query_paperdb does not 
support multiple positional arguments or combined positional and keyword arguments

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("""## LLM 安全相关代表性论文一览（基于 2025 年顶会检索）                                            
                                                                                                                   
  根据数据库检索结果，以下是近年在 USENIX Security、CCS                                                            
  等顶级安全会议上发表的、与大模型（LLM）安全密切相关的论文。为了方便理解，我按**攻击 /                            
  防御类型**进行简单归类，并用通俗中文做简要说明。                                                                 
                                                                                                                   
  ---                                                                                                              
  ## 一、Jailbreak（越狱）攻击与防御                                                                               
                                                                                                                   
  这些工作主要研究：如何绕过 LLM 的安全对齐机制（越狱），以及如何抵御此类攻击。                                    
                                                                                                                   
  ### 1. JBShield: Defending Large Language Models from Jailbreak Attacks through Activated Concept Analysis and   
  Manipulation                                                                                                     
  *会议：USENIX Security 2025*                                                                                     
  - **问题**：现有 LLM 易被「越狱提示词」诱导输出违规内容（暴力、违法、隐私等）。                                  
  -                                                                                                                
  **核心思路**：通过分析模型内部被激活的“概念”表示，识别出与越狱/有害内容相关的激活模式，然后对这些激活进行**抑制  
  或重写**，从而在内部层面阻断越狱效果，而不仅仅依赖提示词规则过滤。                                               
  - **启示**：                                                                                                     
    - 安全防护不应只停留在「输入输出层」黑盒过滤，                                                                 
    - 利用可解释性/概念激活分析，可以在**模型内部**构建更稳健的安全防线。                                          
                                                                                                                   
  ### 2. Exploiting Task-Level Vulnerabilities: An Automatic Jailbreak Attack and Defense Benchmarking for LLMs    
  *会议：USENIX Security 2025*                                                                                     
  - **问题**：现有越狱研究多集中在具体“提示技巧”，缺乏从**任务层面**系统评估不同场景下 LLM 的安全脆弱性。          
  - **核心思路**：                                                                                                 
    - 构建一个**自动化的 jailbreaking 攻击与防御评测平台**，                                                       
    - 从不同任务（如代码生成、问答、对话、翻译等）层面去挖掘 LLM 的漏洞，                                          
    - 系统比较各种攻击和防御机制在不同任务上的表现。                                                               
  - **启示**：                                                                                                     
    - LLM 安全评估要考虑「**任务上下文**」，有的任务天生风险更大（如代码执行、系统操作等）。                       
                                                                                                                   
  ### 3. SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical Manner                         
  *会议：USENIX Security 2025*                                                                                     
  - **问题**：如何在不大幅修改模型架构的前提下，让 LLM 在真实系统中更好地抵御越狱？                                
  - **核心思路**：                                                                                                 
    - 利用 LLM 本身的推理与自我反思能力，让模型**自己检测输入是否包含越狱企图**，                                  
    - 在检测到风险时，自动调整响应策略（拒绝、重写、请求澄清等），                                                 
    - 更偏向实用工程视角，可部署在现有系统中。                                            

Final answer: ## LLM 安全相关代表性论文一览（基于 2025 年顶会检索）

根据数据库检索结果，以下是近年在 USENIX Security、CCS 
等顶级安全会议上发表的、与大模型（LLM）安全密切相关的论文。为了方便理解，我按**攻击 / 
防御类型**进行简单归类，并用通俗中文做简要说明。

---
## 一、Jailbreak（越狱）攻击与防御

这些工作主要研究：如何绕过 LLM 的安全对齐机制（越狱），以及如何抵御此类攻击。

### 1. JBShield: Defending Large Language Models from Jailbreak Attacks through Activated Concept Analysis and 
Manipulation  
*会议：USENIX Security 2025*
- **问题**：现有 LLM 易被「越狱提示词」诱导输出违规内容（暴力、违法、隐私等）。
- 
**核心思路**：通过分析模型内部被激活的“概念”表示，识别出与越狱/有害内容相关的激活模式，然后对这些激活进行**抑制或重
写**，从而在内部层面阻断越狱效果，而不仅仅依赖提示词规则过滤。
- **启示**：
  - 安全防护不应只停留在「输入输出层」黑盒过滤，
  - 利用可解释性/概念激活分析，可以在**模型内部**构建更稳健的安全防线。

### 2. Exploiting Task-Level Vulnerabilities: An Automatic Jailbreak Attack and Defense Benchmarking for LLMs  
*会议：USENIX Security 2025*
- **问题**：现有越狱研究多集中在具体“提示技巧”，缺乏从**任务层面**系统评估不同场景下 LLM 的安全脆弱性。
- **核心思路**：
  - 构建一个**自动化的 jailbreaking 攻击与防御评测平台**，
  - 从不同任务（如代码生成、问答、对话、翻译等）层面去挖掘 LLM 的漏洞，
  - 系统比较各种攻击和防御机制在不同任务上的表现。
- **启示**：
  - LLM 安全评估要考虑「**任务上下文**」，有的任务天生风险更大（如代码执行、系统操作等）。

### 3. SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical Manner  
*会议：USENIX Security 2025*
- **问题**：如何在不大幅修改模型架构的前提下，让 LLM 在真实系统中更好地抵御越狱？
- **核心思路**：
  - 利用 LLM 本身的推理与自我反思能力，让模型**自己检测输入是否包含越狱企图**，
  - 在检测到风险时，自动调整响应策略（拒绝、重写、请求澄清等），
  - 更偏向实用工程视角，可部署在现有系统中。
- **启示**：
  - 「用 LLM 守 LLM」是一个重要趋势，
  - 通过链式思维、自我批评、多轮审查等机制，能在工程上显著增强安全性。

### 4. TwinBreak: Jailbreaking LLM Security Alignments based on Twin Prompts  
*会议：USENIX Security 2025*
- **问题**：即便模型经过安全对齐，仍可通过巧妙设计提示组合进行绕过。
- **核心思路**：
  - 提出 **“Twin Prompts”（双提示）** 概念：利用两个协同的提示，一个负责伪装/引导，另一个负责触发具体的有害行为，
  - 系统研究这类双提示结构如何破坏对齐策略，并评估各类防御机制的效果。
- **启示**：
  - 实际攻击常不是“单条提示”那么简单，而是复杂的对话/多提示结构，
  - 防御设计时也要考虑**会话级别、多轮、多提示**的组合攻击。

### 5. Prompt Obfuscation for Large Language Models  
*会议：USENIX Security 2025*
- **问题**：在开放环境中部署 LLM 时，提示词（尤其是系统提示 / 指令）可能泄露敏感策略或被篡改。
- **核心思路**：
  - 研究如何对提示进行**混淆（obfuscation）**，既让模型仍然理解并执行指令，又让攻击者难以直接读取或逆向出策略，
  - 在提示保护、prompt leak 防护等方向上给出系统性方法。
- **启示**：
  - 除了“内容安全”，还要关注**提示本身的机密性与完整性**（prompt confidentiality & integrity）。

---
## 二、隐私攻击与 Membership Inference

### 6. Towards Label-Only Membership Inference Attack against Pre-trained Large Language Models  
*会议：USENIX Security 2025*
- **问题**：
  - Membership Inference Attack（MIA）试图判断某条数据是否被用来训练模型，
  - 在很多实际场景中，攻击者只拿到“模型输出的标签/回答”，没有概率分布等详细信息。
- **核心思路**：
  - 提出针对 **“仅标签”输出（label-only）** 场景的 MIA 方法，
  - 分析在预训练 LLM 上，这类攻击在何种设定下仍然可行，以及风险程度。
- **启示**：
  - 即使接口极度受限（只返回结果而非置信度），
  - 训练数据隐私仍旧存在泄露风险，
  - 对使用敏感数据（医疗、金融等）训练的大模型，要认真评估 MIA 风险。

### 7. Membership Inference Attacks Against Vision-Language Models  
*会议：USENIX Security 2025*
- **问题**：Vision-Language Models（VLMs）同时处理图像与文本，其训练数据可能包含大量敏感图像（人脸、隐私场景等）。
- **核心思路**：
  - 系统性分析对 VLM 的 membership inference 攻击，
  - 探究在多模态设定下，如何利用模型的图文输出特征判断某张图像/文本是否出现在训练集中。
- **启示**：
  - 多模态大模型的隐私攻击面更大，
  - 需要针对图像、文本及其组合构建新的隐私保护机制（如 DP、多模态加噪等）。

---
## 三、数据与知识投毒（Poisoning）

### 8. PoisonedRAG: Knowledge Corruption Attacks to Retrieval-Augmented Generation of Large Language Models  
*会议：USENIX Security 2025*
- **问题**：
  - RAG（Retrieval-Augmented Generation）系统依赖外部知识库，
  - 如果知识库被恶意污染（加入错误、有害、带后门的信息），LLM 生成内容会被误导甚至执行后门行为。
- **核心思路**：
  - 系统研究 **“知识腐蚀攻击”**：在检索库中注入特定干扰内容，
  - 分析攻击者如何通过少量恶意文档显著改变 RAG 输出，
  - 并探讨潜在的检测与防御方法。
- **启示**：
  - RAG 系统一旦上线，其**检索库安全**极其关键，
  - 需要对知识库进行访问控制、数据源可信度评估、周期性审计与异常检测。

---
## 四、代码生成与软件安全

### 9. Give LLMs a Security Course: Securing Retrieval-Augmented Code Generation via Knowledge Injection  
*会议：CCS 2025*
- **问题**：LLM + RAG 用于代码生成时，模型可能输出不安全的代码（存在漏洞、后门、错误配置）。
- **核心思路**：
  - 通过**注入安全知识**（knowledge injection），在 RAG 流程中显式检索与安全相关的最佳实践、规则、样例，
  - 让模型在生成代码时，参考这些“安全教材”，从而减少典型安全漏洞。
- **启示**：
  - 为 LLM「补安全课」非常重要，
  - 通过工具链、RAG 知识库、静态分析等组合，可显著提升代码生成安全性。

---
## 五、其他安全相关方向

### 10. Password Guessing Using Large Language Models  
*会议：USENIX Security 2025*
- **问题**：评估 LLM 在**密码猜测**与破解场景中的能力与风险。
- **核心思路**：
  - 利用 LLM 的语言建模能力，生成高概率/有结构的密码候选，
  - 研究其在实际攻击中的效用，与传统密码猜测工具（如基于规则、概率模型的工具）对比。
- **启示**：
  - LLM 本身也可能被用作攻击工具（密码猜测、社工邮件撰写、钓鱼内容生成等），
  - 安全研究需要正视“攻防两用”的双刃剑属性。

---
#

[Step 3: Duration 82.06 seconds| Input tokens: 6,974 | Output tokens: 3,355]

## Conclusion

- 数据基座：从 dblp 拉取信安四大会 2025 TOC，用 `OpenAIEmbeddings` 构建并持久化 `FAISS` 向量库，后续各方案共享。

- 方案0（原生 tool call）：直接用 OpenAI `responses.create` + JSON Schema 工具。链路最短、调试透明、可强制 `tool_choice`；但对话状态管理、重试、去重、输出都要手写。

- 方案1（SmolAgent::CodeAgent）：封装规划+多步推理，`use_structured_outputs_internally` 决定输出格式（叙事式 vs JSON）。自带 `@tool` 装饰、流式打印，适合想要自动规划、教学演示或快速接 UI 的场景。

- 方案2（LangGraph）：用显式状态机把 LLM 节点与 `ToolNode` 串起来，`tools_condition` 控制循环；便于插入过滤/重排/裁剪等治理节点，可视化和监控更友好，适合可编排性与合规要求高的流水线。

- 方案3（CodeAgent + MCP）：工具通过 MCP 服务集中托管，前端代理几乎零改动即可复用跨项目/多语言工具；适合团队内统一工具治理、远程扩展或把工具独立部署成服务。

- 选型指南：

  - 快速内嵌或最小依赖：选方案0。

  - 需要自动规划、流式展示或教学：选方案1。

  - 流程复杂、需可视化/治理节点：选方案2。

  - 工具要集中化、跨项目共享或远程调用：选方案3。